In [1]:
from Bio import SeqIO
import mappy as mp

In [8]:
def t(a):
    for i in a:
        i.what = 'ya'
        
b = [{'b': 't'}, {'b': 'e'}]
b[0]['b']

't'

In [30]:
reads = []
for rec in SeqIO.parse('../pLalign/test_data/FAY32610_pass_barcode13_92612969_9c09aa16_0.fastq', "fastq"):
    reads.append(str(rec.seq))

In [31]:
ref_fname = '../pLalign/test_data/pmj09.gb'
record = SeqIO.read(ref_fname, 'genbank')
print(record.seq)

CTGCCCATGTTCTTTCCTGCGTTATCAGAGCTTATCGGCCAGCCTCGCAGAGCAGGATTCCCGTTGAGCACCGCCAGGTGCGAATAAGGGACAGTGAAGAAGGAACACCCGCTCGCGGGTGGGCCTACTTCACCTATCCTGCCCGGCTGACGCCGTTGGATACACCAAGGAAAGTCTACACGAACCCTTTGGCAAAATCCTGTATATCGTGCGAAAAAGGATGGATATACCGAAAAAATCGCTATAATGACCCCGAAGCAGGGTTATGCAGCGGAAAGTATACCTTAAACAATGTTCTTGTGGGCGGACAAAATAGTTGGGAACTGGGAGGGGTGGAAATGGAGTTTTTAAGGATTATTTAGGGAAGAGTGACAAAATAGATGGGAACTGGGTGTAGCGTCGTAAGCTAATACGAAAATTAAAAATGACAAAATAGTTTGGAACTAGATTTCACTTATCTGGTTGGAACGCCCATCTAGCGGTAATACGACTCACTATAGGTCAGGAAATGTGCGCGGAACCAGATAATTCTTTCAGTAGCTTGCAGTGGGCTTACATGGCGATAGCTAGACTGGGCGGTTTTATGGACAGCAAGCGAACCGGAATTGCCAGCTGGGGCGCCCTCTGGTAAGGTTGGGAAGCCCTGCAAAGTAAACTGGATGGCTTTCTTGCCGCCAAGGATCTGATGGCGCAGGGGATCAAGATCTGATCAAGAGACAGGATGAGGATCGTTTCTGATGATTGAACAAGATGGATTGCACGCAGGTTCTCCGGCCGCTTGGGTGGAGAGGCTATTCGGCTATGACTGGGCACAACAGACAATCGGCTGCTCTGATGCCGCCGTGTTCCGGCTGTCAGCGCAGGGGCGCCCGGTTCTTTTTGTCAAGACCGACCTGTCCGGTGCCCTGAATGAACTCCAAGACGAGGCAGCGCGGCTATCGTGGCTGGCCACGACGGGCGTTCCTTGCGCAGCTGTGCTCGACGTTGTCACTGAAGCGGG

In [32]:
alignment_len_thresh = 100
mapq_thresh = 30

def do_alignment(aligner, read_data, stop_point=None):
    i = 0
    all_hits = []
    read_lens = []
    for seq in read_data: #name, seq, qual in mp.fastx_read(io.StringIO(text)):
        read_lens.append(len(seq))
        i += 1
        hits = [h for h in aligner.map(seq)]
        keep_hits = []
        for hit in hits:
            hit_len = hit.r_en-hit.r_st
            if hit_len > alignment_len_thresh and hit.mapq > mapq_thresh:
                hit_excluded = False
                for hit2 in keep_hits:
                    if hit.r_st < hit2['ref_end'] and hit.r_st > hit2['ref_start'] and hit_len < (hit2['ref_end']-hit2['ref_start']):
                        hit_excluded = True
                        print('Excluding smaller overlapping alignment')
                if not hit_excluded:
                    keep_hits.append({
                        'cigar': hit.cigar, 
                        'ref_start': hit.r_st, 
                        'ref_end': hit.r_en, 
                        'strand': hit.strand, 
                        'query_start': hit.q_st, 
                        'query_end': hit.q_en, 
                        'query_seq': seq,
                        'mapq': hit.mapq
                    })
                    
        if len(keep_hits) > 0:
            all_hits.append(keep_hits)
        if stop_point and stop_point < i:
            break
    
    return all_hits, read_lens

ref_seq = record.seq
aligner = mp.Aligner(seq=str(ref_seq)+str(ref_seq))
ah, rl = do_alignment(aligner, reads)

Excluding smaller overlapping alignment
Excluding smaller overlapping alignment
Excluding smaller overlapping alignment


In [34]:
ah[0]

[{'cigar': [[51, 0],
   [1, 2],
   [1, 0],
   [2, 2],
   [41, 0],
   [1, 1],
   [38, 0],
   [1, 1],
   [47, 0],
   [1, 2],
   [217, 0],
   [1, 1],
   [34, 0],
   [2, 1],
   [1, 0],
   [1, 1],
   [3, 0],
   [1, 1],
   [14, 0],
   [2, 2],
   [9, 0],
   [2, 2],
   [2, 0],
   [1, 1],
   [153, 0],
   [1, 2],
   [8, 0],
   [6, 2],
   [60, 0],
   [2, 2],
   [1, 0],
   [1, 2],
   [67, 0],
   [1, 1],
   [88, 0],
   [2, 2],
   [1, 0],
   [1, 2],
   [127, 0],
   [1, 2],
   [431, 0],
   [1, 2],
   [72, 0],
   [1, 2],
   [71, 0],
   [1, 2],
   [247, 0],
   [1, 2],
   [53, 0],
   [1, 1],
   [105, 0],
   [1, 2],
   [248, 0],
   [1, 2],
   [68, 0],
   [1, 2],
   [2, 0],
   [1, 2],
   [3, 0],
   [2, 2],
   [2, 0],
   [3, 1],
   [25, 0],
   [1, 2],
   [2, 0],
   [1, 1],
   [267, 0],
   [1, 1],
   [20, 0],
   [1, 1],
   [24, 0],
   [1, 1],
   [113, 0],
   [2, 1],
   [4, 0],
   [1, 1],
   [3, 0],
   [1, 1],
   [84, 0],
   [1, 1],
   [1, 0],
   [1, 1],
   [243, 0],
   [1, 1],
   [5, 0],
   [1, 2],
   [6, 0

In [33]:
for alignment in ah:
    mismatches = []
    insertions = []
    deletions = []
    alignment_by_ref_pos = [[] for i in range(len(ref_seq))]

    // flipping things around if it is on the other strand
    // (not the simplest way to do this, but it works)
    const use_seq = strand == 1 ? this.refSequence+this.refSequence : this.refSequenceRC+this.refSequenceRC;
    const ref_start = strand == 1 ? alignment.ref_start : this.refLen*2-alignment.ref_end;
    const use_cigar = strand == 1 ? alignment.cigar : alignment.cigar.slice().reverse();
    if strand == 1:
      query_position = alignment.query_start
    else:
      query_position = alignment.que
    ref_position = ref_start;
    let mismatches = [];
    let deletions = [];
    let insertions = [];
    let alignment_by_ref_pos = [];
    for (let i=0; i<this.refLen; i++) {
      alignment_by_ref_pos.push([]);
    }
    if (query_position > 0) {
      for (let i=0; i<query_position; i++) {
        alignment_by_ref_pos[this.refpos(strand, ref_position-query_position+i)].push(['pre-alignment', this.qrc(strand, query[i])]);
      }
    }
    for (let cig of use_cigar) {
      let bp = cig[0];
      let letter = "MIDNSHP=XB"[cig[1]];
      if (letter == 'S') {
        for (let i=0; i<bp; i++) {
          alignment_by_ref_pos[this.refpos(strand, ref_position+i)].push(['pre-alignment', '+', this.qrc(strand, query[query_position+i])]);
        }
        query_position += bp;
      } else if (letter == 'M') {
        for (let i=0; i<bp; i++) {
          if (query[query_position+i]!=use_seq[ref_position+i]) {
            mismatches.push(ref_position+i);
            alignment_by_ref_pos[this.refpos(strand, ref_position+i)].push(['mismatch', this.qrc(strand, query[query_position+i])]);
          } else {
            alignment_by_ref_pos[this.refpos(strand, ref_position+i)].push(['match', this.qrc(strand, query[query_position+i])]);
          }
        }
        query_position += bp;
        ref_position += bp;
      } else if (letter == 'D') {
        for (let i=0; i<bp; i++) {
          alignment_by_ref_pos[this.refpos(strand, ref_position+i)].push(['deletion', '-']);
        }
        ref_position += bp;
        deletions.push([ref_position-bp, ref_position]);
      } else if (letter == 'I') {
        alignment_by_ref_pos[this.refpos(strand, ref_position)].push(['insertion', '', this.qrc(strand, query.slice(query_position, query_position+bp))]);
        query_position += bp;
        insertions.push([ref_position, bp]);
      }
    }
    if (query_position < query.length-1) {
      console.log('Q', query.length, query_position);
      for (let i=0; i<query.length-1-query_position; i++) {
        alignment_by_ref_pos[this.refpos(strand, ref_position+i)].push(['post-alignment', this.qrc(strand, query[query_position+i])]);
      }
    }
    console.log(ref_start, ref_position, ref_start - ref_position, strand, mismatches.length);
    //console.log(mismatches, deletions, insertions);
    let al_g = this.svg.append('g')
      .attr('class', 'alignment_group');

    let al_color = colors['al'];
    if (Math.abs(ref_start-ref_position) > this.refLen) {
      console.log('alignment longer than reference...');
      // dealing with cases where the alignment is longer than the ref...
      al_g.append('path')
        .attr('class', 'alignment_path')
        .attr('fill', 'none')
        .attr('stroke', colors['al'])
        .attr('stroke-width', this.align_r/2)
        .attr('d', self.make_alignment_arc(ref_start, ref_start+(this.refLen-1)*strand, this.ref_r+this.feature_h+alignment_count*this.align_r, strand));

      ref_position = ref_position-this.refLen;
      al_color = colors['overlap'];
    }

    al_g.append('path')
      .attr('class', 'alignment_path')
      .attr('fill', 'none')
      .attr('stroke', al_color)
      .attr('stroke-width', this.align_r/2)
      .attr('d', self.make_alignment_arc(ref_start*strand, ref_position*strand, this.ref_r+this.feature_h+alignment_count*this.align_r, strand));

    al_g.selectAll('.mismatch_marks')
      .data(mismatches)
      .enter()
      .append('path')
        .attr('class', 'mismatch_marks')
        .attr('fill', 'none')
        .attr('stroke', colors['X'])
        .attr('stroke-width', this.align_r/2)
        .attr('d', (d) => self.make_alignment_arc(d*strand-0.5, d*strand+0.5, this.ref_r+this.feature_h+alignment_count*this.align_r, strand));

    al_g.selectAll('.deletion_arcs')
      .data(deletions)
      .enter()
      .append('path')
        .attr('class', 'deletion_arcs')
        .attr('fill', 'none')
        .attr('stroke', colors['D'])
        .attr('stroke-width', this.align_r/2)
        .attr('d', (d) => self.make_alignment_arc(d[0]*strand-0.5, d[1]*strand+0.5, this.ref_r+this.feature_h+alignment_count*this.align_r, strand));
    
    al_g.selectAll('.insertion_arcs')
      .data(insertions)
      .enter()
      .append('path')
        .attr('class', 'insertion_arcs')
        .attr('fill', 'none')
        .attr('stroke', colors['I'])
        .attr('stroke-width', this.align_r/3)
        .attr('d', (d) => self.make_alignment_arc((d[0]-d[1]/2)*strand-0.5, (d[0]+d[1]/2)*strand+0.5, this.ref_r+this.feature_h+alignment_count*this.align_r+this.align_r/4, strand));

    return alignment_by_ref_pos
  }

  draw_alignments() {
    d3.selectAll('.alignment_group').remove();
    this.base_alignments = [];
    for (let i=alignment_num*this.alignment_page; 
         i<Math.min(alignment_num*(1+this.alignment_page), this.alignments.length); i++) {
      for (let a of this.alignments[i]) { 
        // alignments are in a list in case there are multiple per query (rare)
        this.base_alignments.push(this.draw_alignment(a, i-alignment_num*this.alignment_page));
      }
    }
    console.log(this.base_alignments);
  }

[{'cigar': [[51, 0],
   [1, 2],
   [1, 0],
   [2, 2],
   [41, 0],
   [1, 1],
   [38, 0],
   [1, 1],
   [47, 0],
   [1, 2],
   [217, 0],
   [1, 1],
   [34, 0],
   [2, 1],
   [1, 0],
   [1, 1],
   [3, 0],
   [1, 1],
   [14, 0],
   [2, 2],
   [9, 0],
   [2, 2],
   [2, 0],
   [1, 1],
   [153, 0],
   [1, 2],
   [8, 0],
   [6, 2],
   [60, 0],
   [2, 2],
   [1, 0],
   [1, 2],
   [67, 0],
   [1, 1],
   [88, 0],
   [2, 2],
   [1, 0],
   [1, 2],
   [127, 0],
   [1, 2],
   [431, 0],
   [1, 2],
   [72, 0],
   [1, 2],
   [71, 0],
   [1, 2],
   [247, 0],
   [1, 2],
   [53, 0],
   [1, 1],
   [105, 0],
   [1, 2],
   [248, 0],
   [1, 2],
   [68, 0],
   [1, 2],
   [2, 0],
   [1, 2],
   [3, 0],
   [2, 2],
   [2, 0],
   [3, 1],
   [25, 0],
   [1, 2],
   [2, 0],
   [1, 1],
   [267, 0],
   [1, 1],
   [20, 0],
   [1, 1],
   [24, 0],
   [1, 1],
   [113, 0],
   [2, 1],
   [4, 0],
   [1, 1],
   [3, 0],
   [1, 1],
   [84, 0],
   [1, 1],
   [1, 0],
   [1, 1],
   [243, 0],
   [1, 1],
   [5, 0],
   [1, 2],
   [6, 0